In [3]:
%%capture 
!python3 -m pip install google.cloud
!python3 -m pip install pandas
!python3 -m pip install numpy
!python3 -m pip install pyarrow

In [4]:
import os
import pandas as pd
import numpy as np
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.bigquery import magics
magics.context.use_bqstorage_api = True
import pyarrow
from sklearn.linear_model import LogisticRegression
from matplotlib import pyplot as plt
import random

In [5]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
np.set_printoptions(suppress=True, linewidth=200, edgeitems=100)

# Pipeline and column transformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer

# Data transformers
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, label_binarize
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

# Data splitter and model evaluator
from sklearn.model_selection import train_test_split, validation_curve, learning_curve, GridSearchCV

# Learning models
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import BaggingClassifier, BaggingRegressor, GradientBoostingClassifier, GradientBoostingRegressor
#from xgboost import XGBClassifier, XGBRegressor  # Need to install

# Performance metrics
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

In [ ]:
# CONFIGURE THE BIGQUERY SETTINGS

BIGQUERY_PROJECT = 'ironhacks-covid19-data'
BIGQUERY_KEYPATH = 'service-account.json'

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = BIGQUERY_KEYPATH
bigquery_client = bigquery.Client(project=BIGQUERY_PROJECT)

In [ ]:
query = """
SELECT poi_id, top_category, week_number, raw_visit_counts, postal_code
FROM ironhacks_covid19_competition.weekly_patterns
"""
query_job = bigquery_client.query(query)
wp = query_job.to_dataframe()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:441: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


In [ ]:
query = """
SELECT week_number, cases
FROM ironhacks_covid19_competition.covid19_cases
"""
query_job = bigquery_client.query(query)
covid = query_job.to_dataframe()

In [ ]:
#shifting the covid_cases up one and two weeks to decrease the delay in people's reaction
# nc = covid.copy()
# nc['cases_shift1'] = nc['cases'].shift(1)
# nc['cases_shift2'] = nc['cases'].shift(2)
# del nc['cases']
# nc = nc.fillna(7)
# new = wp.merge(nc, on = 'week_number')
new = wp


In [ ]:
dummy = pd.get_dummies(new['top_category'])
table = new.join(dummy)
dummy = pd.get_dummies(new['poi_id'])
table = table.join(dummy)
# dummy = pd.get_dummies(new['postal_code'])
# table = table.join(dummy)
del table['top_category']
del table['poi_id']
del table['postal_code']

In [ ]:
table

,week_number,raw_visit_counts,"Accounting, Tax Preparation, Bookkeeping, and Payroll Services",Activities Related to Credit Intermediation,Administration of Economic Programs,"Agencies, Brokerages, and Other Insurance Related Activities",Amusement Parks and Arcades,Automobile Dealers,Automotive Equipment Rental and Leasing,"Automotive Parts, Accessories, and Tire Stores",...,fe190b44-532c-4f05-bd5e-64a819f37987,fe227e74-6422-4564-9c10-470fd26b9d6c,fe52a09d-6b37-4f62-8613-bc1c0eda7b55,feb97e54-f336-4440-9ea0-0b29ef024142,fef4f7b8-aba7-4740-ad71-6af091dd071c,ff017941-e5c5-4c3e-ad8e-51bee7a0d3f3,ff047527-176f-4d9b-9d91-0d0ab8805a44,ffa5b55e-5f61-4cd7-8f66-741f492c34c6,ffdda836-f200-470e-9e9d-c8917a164043,fffcc243-6692-466d-a427-4696a2f7ab61
0,11,23,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,11,9,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,11,11,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,11,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,11,22,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51726,43,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51727,43,23,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51728,43,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51729,43,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
filt_wp = new[new['week_number'] == 43]
dummy = pd.get_dummies(filt_wp['top_category'])
table2 = filt_wp.join(dummy)
dummy = pd.get_dummies(filt_wp['poi_id'])
table2 = table2.join(dummy)
dummy = pd.get_dummies(filt_wp['postal_code'])
table2 = table2.join(dummy)
del table2['top_category']
del table2['raw_visit_counts']
del table2['postal_code']

table2['week_number'] = table2['week_number'].replace(43, 44)
# table2['cases_shift1'] = table2['cases_shift1'].replace(25872, 29641)
# table2['cases_shift2'] = table2['cases_shift2'].replace(23218, 25872)

In [ ]:
# df = table  
# # df.head()

# X = df.drop(['raw_visit_counts'], axis=1)
# y = df['raw_visit_counts']


In [ ]:
df = table
X = df.drop(['raw_visit_counts'], axis=1)
y = df['raw_visit_counts']
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.3, 
                                                    random_state=0)

In [ ]:
# ridge_model = Ridge(alpha = 0.1)
# ridge_model.fit(X_train, y_train)
# print('Ridge Training score:', ridge_model.score(X_train,y_train))
# print('Ridge Test score:    ', ridge_model.score(X_test,y_test),'\n')

# lasso_model = Lasso(alpha = 0.1)
# lasso_model.fit(X_train, y_train)
# print('Lasso Training score:', lasso_model.score(X_train,y_train))
# # print('Lasso Test score:    ', lasso_model.score(X_test,y_test),'\n')

# decision_tree_model = DecisionTreeRegressor(random_state=0)
# decision_tree_model.fit(X_train, y_train)
# print('Decision Tree Training score:', decision_tree_model.score(X_train,y_train))
# print('Decision Tree Test score:    ', decision_tree_model.score(X_test,y_test),'\n')

random_forest_model = RandomForestRegressor(random_state=0)
random_forest_model.fit(X_train, y_train)
print('Random Forest Training score:', random_forest_model.score(X_train,y_train))
print('Random Forest Test score:    ', random_forest_model.score(X_test,y_test),'\n')

# gradient_boosting_model = GradientBoostingRegressor(random_state=0)
# gradient_boosting_model.fit(X_train, y_train)
# print('Gradient Boosting Training score:', gradient_boosting_model.score(X_train,y_train))
# print('Gradient Bosting Test score:    ', gradient_boosting_model.score(X_test,y_test),'\n')

Gradient Boosting Training score: 0.7205289206892517
Gradient Bosting Test score:     0.6757111139108268 



In [19]:
# cross validation (takes a long time so I commented it out)
# scores = cross_val_score(estimator=decision_tree_model, X=X, y=y, cv=10)
# print(scores)

[0.94814607 0.94581731 0.92372189 0.97262571 0.97723532 0.90800445 0.68370785 0.96692202 0.97305662 0.95705443]


In [33]:
X_predict = table2
poi_id = table2['poi_id']
del X_predict['poi_id']
raw_visit_counts = random_forest_model.predict(X_predict)


d = pd.DataFrame(columns = ['poi_id', 'raw_visit_counts'])
d['poi_id'] = poi_id
d['raw_visit_counts'] = raw_visit_counts


# d.to_csv('prediction3.csv',index=False)